# 🍽️ Restaurant Recommendation System (Google Colab Project)
This notebook builds a **content-based recommendation system** to suggest restaurants based on user preferences.

In [ ]:
# Step 1: Upload Dataset
from google.colab import files
uploaded = files.upload()

import pandas as pd
data = pd.read_csv("Datasetml.csv")
print(data.head())
print(data.columns)
print(data.info())
print(data.isnull().sum())

In [ ]:
# Step 2: Preprocessing
from sklearn.preprocessing import LabelEncoder

# Handle missing values
for col in data.columns:
    if data[col].dtype == 'object':
        data[col].fillna(data[col].mode()[0], inplace=True)
    else:
        data[col].fillna(data[col].mean(), inplace=True)

# Encode categorical columns
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

data.head()

In [ ]:
# Step 3: Compute Similarity Matrix
from sklearn.metrics.pairwise import cosine_similarity

# Choose features for recommendation (adjust based on dataset)
features = ["Cuisines", "Price range", "Country Code"]
X = data[features]

# Compute similarity
similarity = cosine_similarity(X)
print("Similarity matrix computed with shape:", similarity.shape)

In [ ]:
# Step 4: Recommendation Function
def recommend_restaurants(user_pref, top_n=5):
    """
    user_pref: dict with keys like {"Cuisines": "Italian", "Price range": 3, "Country Code": 1}
    top_n: number of recommendations
    """
    # Encode user preferences
    user_vector = []
    for col in features:
        if col in label_encoders:  # categorical
            user_vector.append(label_encoders[col].transform([user_pref[col]])[0])
        else:
            user_vector.append(user_pref[col])

    user_vector = pd.DataFrame([user_vector], columns=features)
    
    # Compute similarity
    user_similarity = cosine_similarity(user_vector, X)[0]

    # Top recommendations
    indices = user_similarity.argsort()[-top_n:][::-1]
    return data.iloc[indices][["Restaurant Name", "Cuisines", "Price range", "Aggregate rating"]]

In [ ]:
# Step 5: Test the Recommendation System
user_pref = {"Cuisines": "Italian", "Price range": 3, "Country Code": 1}
recommendations = recommend_restaurants(user_pref, top_n=5)
print("Recommended Restaurants:")
print(recommendations)